<img src="https://gallery.mailchimp.com/f98d5ac0a3fbbdcdda35136ab/images/2002af76-5fd4-4185-9d49-28558b6b8772.png">

# `sg-hdb-resale` 
# Part 1: Extract, Transform, Load

In this notebook, I will be carry out steps and constructing structures that allows the following:
+ preliminary exploration of the raw data in terms of its state and format that it comes in
+ storing of extracted data in variables
+ loading extracted data into a database

These things takes one through the **ETL** process that is facilitated by a set of functions that makes a data pipeline. Albeit a simple one, beginners can learn from this whole process that we're about to go through.

<img src="https://i.ibb.co/wJQ4fK7/etl-workflow-image.png">

What is **ETL**?
+ **Extract:** This is the process of extracting data/information from the raw files. In our context here, the raw files have been provided to us in CSV form. In other enterprise use cases these raw files can come in other forms such as streamed JSON objects or transactional data from OLTP databases.
+ **Transform:** The process of converting data from the aforementioned extraction process to a digestible format to be ingested to another database or a datalake.
+ **Load:** Following transformation where the extracted data has been reformated, the process of loading all of it into a database comes under here.

In [1]:
# Begin by importing the packages we need
import os
import pandas as pd

<font color="blue"><h1><center>Extract</center></h1></font>
Download data at:
+ https://data.gov.sg/dataset/resale-flat-prices
+ https://data.gov.sg/dataset/hdb-resale-price-index

__Resale Flat Prices:__ This dataset consist of transactions for HDB resale units.
__Resale Price Index:__ Tracks the overall price movement of the public residential market.

This section is where we carry out the main objective of extracting the data from the CSV files containing information regarding HDB resale units. First, let us navigate and inspect the folder containing the relevant datasets.

In [2]:
# Listing down the list of files in the relevant directory
# Where prefixed with `!`, a shell runs the command

# Run the following if you're running on Mac
# !ls ../data/raw/resale-flat-prices/

# Run the following if you're running on Windows
# !dir ..\data\raw\resale-flat-prices\

metadata-resale-flat-prices.txt
resale-flat-prices-based-on-approval-date-1990-1999.csv
resale-flat-prices-based-on-approval-date-2000-feb-2012.csv
resale-flat-prices-based-on-registration-date-from-jan-2015-onwards.csv
resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv


Now that we have identified the relevant directory and the files within it, let us create a list containing the names of the .csv files within that same directory and folder.

In [3]:
# Set destination of folder containing raw data
file_loc = '../data/raw/resale-flat-prices/'
# This variable will contain every file in folder
file_list = os.listdir(file_loc)
# This variable only bothers with files ending with '.csv'
file_list_csv = [i for i in file_list if i.endswith('.csv')]
print(file_list_csv)

['resale-flat-prices-based-on-registration-date-from-jan-2015-onwards.csv', 'resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv', 'resale-flat-prices-based-on-approval-date-1990-1999.csv', 'resale-flat-prices-based-on-approval-date-2000-feb-2012.csv']


The function that we will be creating below is to combine all the data from all the .csv files into one `pandas` dataframe. We will be using the `pandas` function `concat` to combine. Where a dataset does not have a certain variable that exists in the other dataset, the variable will be retained while filling in '0's for empty values. This is specifically referring to the variable `remaining_lease`.

In [4]:
def CombineDF(file_loc, file_list):
    """
    This function imports all the .csv files and concatenate them together.
    Values for mismatched columns will be filled in with 0s.
    """     
    dataset_files = []
    # This part gives you the location for each .csv file
    for i in file_list:
        dataset_files.append('{}{}'.format(file_loc, i))
        
    print('The files that we will be importing:\n{}'.format(dataset_files))
    
    frames = [ pd.read_csv(f) for f in list(dataset_files) ]
    # .fillna() is being used below as other CSV files does not have `remaining_lease` col
    combi_result = pd.concat(frames, ignore_index=True).fillna(0)
    return combi_result

In [5]:
hdb_combi_df = CombineDF(file_loc, file_list_csv)

The files that we will be importing:
['../data/raw/resale-flat-prices/resale-flat-prices-based-on-registration-date-from-jan-2015-onwards.csv', '../data/raw/resale-flat-prices/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv', '../data/raw/resale-flat-prices/resale-flat-prices-based-on-approval-date-1990-1999.csv', '../data/raw/resale-flat-prices/resale-flat-prices-based-on-approval-date-2000-feb-2012.csv']


/Users/Ryzal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  from ipykernel import kernelapp as app


In [6]:
# Printing the first 5 observations of dataframe
hdb_combi_df.head()

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,174,Improved,3 ROOM,60.0,1986,2015-01,70.0,255000.0,07 TO 09,ANG MO KIO AVE 4,ANG MO KIO
1,541,New Generation,3 ROOM,68.0,1981,2015-01,65.0,275000.0,01 TO 03,ANG MO KIO AVE 10,ANG MO KIO
2,163,New Generation,3 ROOM,69.0,1980,2015-01,64.0,285000.0,01 TO 03,ANG MO KIO AVE 4,ANG MO KIO
3,446,New Generation,3 ROOM,68.0,1979,2015-01,63.0,290000.0,01 TO 03,ANG MO KIO AVE 10,ANG MO KIO
4,557,New Generation,3 ROOM,68.0,1980,2015-01,64.0,290000.0,07 TO 09,ANG MO KIO AVE 10,ANG MO KIO


In [7]:
hdb_combi_df

,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,174,Improved,3 ROOM,60.0,1986,2015-01,70.0,255000.0,07 TO 09,ANG MO KIO AVE 4,ANG MO KIO
1,541,New Generation,3 ROOM,68.0,1981,2015-01,65.0,275000.0,01 TO 03,ANG MO KIO AVE 10,ANG MO KIO
2,163,New Generation,3 ROOM,69.0,1980,2015-01,64.0,285000.0,01 TO 03,ANG MO KIO AVE 4,ANG MO KIO
3,446,New Generation,3 ROOM,68.0,1979,2015-01,63.0,290000.0,01 TO 03,ANG MO KIO AVE 10,ANG MO KIO
4,557,New Generation,3 ROOM,68.0,1980,2015-01,64.0,290000.0,07 TO 09,ANG MO KIO AVE 10,ANG MO KIO
5,603,New Generation,3 ROOM,67.0,1980,2015-01,64.0,290000.0,07 TO 09,ANG MO KIO AVE 5,ANG MO KIO
6,709,New Generation,3 ROOM,68.0,1980,2015-01,64.0,290000.0,01 TO 03,ANG MO KIO AVE 8,ANG MO KIO
7,333,New Generation,3 ROOM,68.0,1981,2015-01,65.0,293000.0,01 TO 03,ANG MO KIO AVE 1,ANG MO KIO
8,109,New Generation,3 ROOM,67.0,1978,2015-01,62.0,300000.0,01 TO 03,ANG MO KIO AVE 4,ANG MO KIO
9,564,New Generation,3 ROOM,68.0,1985,2015-01,69.0,307500.0,13 TO 15,ANG MO KIO AVE 3,ANG MO KIO


Now, we will import the data containing the quarterly HDB resale price index.

In [8]:
hdb_rpi = pd.read_csv('../data/raw/hdb-resale-price-index/housing-and-development-board-resale-price-index-1q2009-100-quarterly.csv')
hdb_rpi

,quarter,index
0,1990-Q1,24.3
1,1990-Q2,24.4
2,1990-Q3,25.0
3,1990-Q4,24.7
4,1991-Q1,24.9
5,1991-Q2,25.5
6,1991-Q3,25.2
7,1991-Q4,25.1
8,1992-Q1,25.8
9,1992-Q2,27.4


Looks like all is good with the imported dataset for HDB resale price indexes. We will not bother with it for the following transformation process.

<font color="blue"><h1><center>Transform</center></h1></font>

The data that we have extracted from the CSV files are quite clean and hence we can choose to not do any transformation prior to the loading process. Of course, in the real world, hardly ever do we get such luck.
Further transformations for the purpose of feature engineering can be implemented during the [modelling phase](./sg-hdb-part2-modelling-reference.ipynb).

Even though the formatting/state of the dataset is good enough for us to ingest into a database, for the purpose of this exercise, let us transform the values of a single variable.

Currently, as seen below, the variable `flat_model` contains many (35) different categories and some are mismatched.

In [9]:
# Display unique values for the variable
hdb_combi_df['flat_model'].unique()

array(['Improved', 'New Generation', 'Model A', 'Standard', 'Simplified',
       'Premium Apartment', 'Maisonette', 'Apartment', 'Model A2',
       'Type S1', 'Type S2', 'Adjoined flat', 'Terrace', 'DBSS',
       'Model A-Maisonette', 'Premium Maisonette', 'Multi Generation',
       'Premium Apartment Loft', 'Improved-Maisonette', '2-room',
       'Premium Apartment.', 'IMPROVED', 'NEW GENERATION', 'STANDARD',
       'MODEL A', 'SIMPLIFIED', 'MODEL A-MAISONETTE', 'APARTMENT',
       'MAISONETTE', 'ADJOINED FLAT', 'TERRACE', '2-ROOM',
       'IMPROVED-MAISONETTE', 'MULTI GENERATION', 'PREMIUM APARTMENT'],
      dtype=object)

In [10]:
# Display no. of categories for the variable
len(hdb_combi_df['flat_model'].unique())

35

We have many different categories but some of them are linked to the same one category and are actually just spelled differently due to entry methods. For example we have the following categories as observed from above:
+ 'Model A'
+ 'MODEL A'

Both are pertaining to a single model category but due to the different casings they are treated as different categories. A simple act of transformation that we can employ is to just convert every letter of the values in the `flat_model` column to lowercase.

In [11]:
hdb_combi_df['flat_model'] = hdb_combi_df['flat_model'].str.lower()

In [12]:
# Display unique values for the column
hdb_combi_df['flat_model'].unique()

array(['improved', 'new generation', 'model a', 'standard', 'simplified',
       'premium apartment', 'maisonette', 'apartment', 'model a2',
       'type s1', 'type s2', 'adjoined flat', 'terrace', 'dbss',
       'model a-maisonette', 'premium maisonette', 'multi generation',
       'premium apartment loft', 'improved-maisonette', '2-room',
       'premium apartment.'], dtype=object)

In [13]:
# Display no. of categories for the variable after transformation
len(hdb_combi_df['flat_model'].unique())

21

As you can see, through transformation we are able to handle mismatched categories and in this sense, we have only done some form of preliminary data cleaning but that of course does not deviate from the essence of the transformation process.

Let us now export the extracted data to one single .csv file for checkpoint.

In [14]:
hdb_combi_df.to_csv("../data/interim/sg-resale-flat-prices-1990-to-2019-jan.csv", index = False)

In [15]:
# Include index for dataframe and renaming the column to 'id'
# The function reset_index 
hdb_combi_df.reset_index(level=0, inplace=True)
hdb_combi_df.head()

,index,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,0,174,improved,3 ROOM,60.0,1986,2015-01,70.0,255000.0,07 TO 09,ANG MO KIO AVE 4,ANG MO KIO
1,1,541,new generation,3 ROOM,68.0,1981,2015-01,65.0,275000.0,01 TO 03,ANG MO KIO AVE 10,ANG MO KIO
2,2,163,new generation,3 ROOM,69.0,1980,2015-01,64.0,285000.0,01 TO 03,ANG MO KIO AVE 4,ANG MO KIO
3,3,446,new generation,3 ROOM,68.0,1979,2015-01,63.0,290000.0,01 TO 03,ANG MO KIO AVE 10,ANG MO KIO
4,4,557,new generation,3 ROOM,68.0,1980,2015-01,64.0,290000.0,07 TO 09,ANG MO KIO AVE 10,ANG MO KIO


In [16]:
# Renaming the 'index' column
hdb_combi_df.columns.values[0] = 'id'

In [17]:
hdb_combi_df.head()

,id,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,0,174,improved,3 ROOM,60.0,1986,2015-01,70.0,255000.0,07 TO 09,ANG MO KIO AVE 4,ANG MO KIO
1,1,541,new generation,3 ROOM,68.0,1981,2015-01,65.0,275000.0,01 TO 03,ANG MO KIO AVE 10,ANG MO KIO
2,2,163,new generation,3 ROOM,69.0,1980,2015-01,64.0,285000.0,01 TO 03,ANG MO KIO AVE 4,ANG MO KIO
3,3,446,new generation,3 ROOM,68.0,1979,2015-01,63.0,290000.0,01 TO 03,ANG MO KIO AVE 10,ANG MO KIO
4,4,557,new generation,3 ROOM,68.0,1980,2015-01,64.0,290000.0,07 TO 09,ANG MO KIO AVE 10,ANG MO KIO


<font color="blue"><h1><center>Load</center></h1></font>

Following extraction and transformation, we now intend to load the data derived from the above processes into a simple [SQLite](https://www.sqlite.org/index.html) RDMS/database. 
(For simplicity's sake, we'll use SQLite for now. In the future, one might want to take a look into remote alternatives.)

In [18]:
# Check max length of a value in a column of object data type
hdb_combi_df.town.str.len().max()

15

In [19]:
# Check number of null values across all columns
hdb_combi_df.isnull().sum()

id                     0
block                  0
flat_model             0
flat_type              0
floor_area_sqm         0
lease_commence_date    0
month                  0
remaining_lease        0
resale_price           0
storey_range           0
street_name            0
town                   0
dtype: int64

# SQLite

In [20]:
import os
import sys
import sqlalchemy
from sqlalchemy import Table, Column, Integer, String, Float, Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

Quick observation on how the datasets that we intend to load into the database looks like.

In [21]:
hdb_combi_df.head()

,id,block,flat_model,flat_type,floor_area_sqm,lease_commence_date,month,remaining_lease,resale_price,storey_range,street_name,town
0,0,174,improved,3 ROOM,60.0,1986,2015-01,70.0,255000.0,07 TO 09,ANG MO KIO AVE 4,ANG MO KIO
1,1,541,new generation,3 ROOM,68.0,1981,2015-01,65.0,275000.0,01 TO 03,ANG MO KIO AVE 10,ANG MO KIO
2,2,163,new generation,3 ROOM,69.0,1980,2015-01,64.0,285000.0,01 TO 03,ANG MO KIO AVE 4,ANG MO KIO
3,3,446,new generation,3 ROOM,68.0,1979,2015-01,63.0,290000.0,01 TO 03,ANG MO KIO AVE 10,ANG MO KIO
4,4,557,new generation,3 ROOM,68.0,1980,2015-01,64.0,290000.0,07 TO 09,ANG MO KIO AVE 10,ANG MO KIO


In [22]:
hdb_rpi.head()

,quarter,index
0,1990-Q1,24.3
1,1990-Q2,24.4
2,1990-Q3,25.0
3,1990-Q4,24.7
4,1991-Q1,24.9


For the next few cells, we will be working towards creating the database:
+ create engine to initialise connection
+ specify table names and their columns

In [23]:
# Create engine
engine = create_engine('sqlite:///../data/processed/sg_hdb.db')
Base = declarative_base()

In [24]:
# Specify properties for tables
class HDBRes(Base):
    # Specifying the table name
    __tablename__ = 'sg_hdb_resale'
    
    # Specifying the column headings and types for this table
    id = Column(Integer, primary_key=True)
    block = Column(String(7))
    flat_model = Column(String(30))
    flat_type = Column(String(20))
    floor_area_sqm = Column(Float())
    lease_commence_date = Column(Integer())
    month = Column(String(7))
    remaining_lease = Column(Integer())
    resale_price = Column(Float())
    storey_range = Column(String(15))
    street_name = Column(String(50))
    town = Column(String(20))
    
class HDBPI(Base):
    # Specifying the table name
    __tablename__ = 'sg_hdb_pi'
    
    # Specifying the column headings and types for this table
    quarter = Column(String(7), primary_key=True)
    index = Column(Float())

In [25]:
# Create tables as defined above
Base.metadata.create_all(engine)

Here, we create a function that allows us to connect with the database created from above and insert values from relevant `pandas` dataframes into the SQLite database.

In [26]:
def SGHDBBulkInsert(table_name, df_to_insert, engine_loc):
    engine = create_engine(engine_loc)
    
    # The orient='records' is the key of this, it allows to align with the format mentioned in the doc to insert in bulks.
    list_to_write = df_to_insert.to_dict(orient='records')
    metadata = sqlalchemy.schema.MetaData(bind=engine)
    table = sqlalchemy.Table(table_name, metadata, autoload=True)
    
    # Open the session
    Session = sessionmaker(bind=engine)
    session = Session()
    
    conn = engine.connect()
    # Insert the dataframe into the database in one bulk
    conn.execute(table.insert(), list_to_write)
    # Commit the changes
    session.commit()
    # Close the session
    session.close()

In [27]:
# Executing insertion of the HDB Resale data
SGHDBBulkInsert('sg_hdb_resale', hdb_combi_df, 'sqlite:///../data/processed/sg_hdb.db')

In [28]:
# Now for the HDB Resale Price Indexes
SGHDBBulkInsert('sg_hdb_pi', hdb_rpi, 'sqlite:///../data/processed/sg_hdb.db')

To observe as to whether the intended operations have been executed successfully, we can use relevant GUI tools to examine the contents of databases. For SQLite, we can use [DB Browser for SQLite](https://sqlitebrowser.org/dl/). Once we have loaded the relevant data into our database, it is time for us to work on a simple machine learning model. On to the next part [here](./sg-hdb-part2-modelling-reference.ipynb)!